In [1]:
# all codes come are changed based on LDR code.
# 用添加噪声的数据进行训练
import argparse
import torch.optim as optim
import os
import yaml
from mcrgan.default import _C as config
from mcrgan.default import update_config
import torch
from mcrgan.trainer import MCRTrainer
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.models import get_BNN_models
from mcrgan.models import get_noise
from mcrgan.models import get_noise_uniform
from mcrgan.models import DiscriminatorMNIST
from mcrgan.models import weights_init_mnist_model_noise

import numpy as np
import torch
import argparse
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.default import _C as config
from mcrgan.default import update_config
from utils.utils import sort_dataset, compute_accuracy, extract_features
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import os
from mcrgan.loss import MCRGANloss
import random
random.seed(0)

In [2]:
# 导入mnist数据集，进行对比
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
DATASET = 'mnist'
ROOT    = 'pth/to/the/dataset'
BATCH_SIZE = 2048
WORKERS = 8
DATA_SAMPLE = 50000
LABEL = 0

dataloader, dataset = get_dataloader(
        data_name=DATASET,
        root=ROOT,
        batch_size=BATCH_SIZE,
        num_workers=WORKERS)

In [3]:
import collections

def get_netG(netD,netV,net_noise,radius=1):
    models = [netD, netV,net_noise]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    fed_state_dict = collections.OrderedDict()
    for key in weight_keys:
        fed_state_dict[key] = worker_state_dict[0][key] + radius*worker_state_dict[2][key]*torch.log(1+torch.exp(worker_state_dict[1][key]))
    return fed_state_dict

# new netG already optimized by adam and time learning rate, so we only using netG to update netV
def update_netV(netV,netG,net_noise,fed_state_dict,radius=1):
    models =[netV,netG,net_noise]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    update_dict = collections.OrderedDict()
    for key in weight_keys:
        update_dict[key] = worker_state_dict[0][key] - (worker_state_dict[1][key]-fed_state_dict[key])*(worker_state_dict[2][key]*radius)/torch.log(1+torch.exp(-worker_state_dict[0][key]))
    return update_dict

# generate different distribution via gaussian init and uniform
def get_cauchy_noise(netD,net_noise1,net_noise2):
    models =[netD,net_noise1, net_noise2]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    update_dict = collections.OrderedDict()
    for key in weight_keys:
        update_dict[key] = worker_state_dict[1][key] / worker_state_dict[2][key]
    return update_dict
        

In [4]:
#models
# train hyperparameters, batch norm not used var
#store_Var = []
import time
from torch_mimicry.training import scheduler, logger, metric_log

from test_acc import nearsub
import numpy as np
import torch
import argparse
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.default import _C as config
from mcrgan.default import update_config
from utils.utils import sort_dataset, compute_accuracy, extract_features2
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import os

start_time = time.time()
#global_step = 0
epsilon_noise = 0.1
num_steps = 1000
n_dis = 1
n_dis2 = 1
numclasses =10
lr1 = 1e-3
lr2 = 1e-3
log_data = metric_log.MetricLog()
data_time = time.time()
iter_dataloader = iter(dataloader)
data, label = next(iter_dataloader)
data_time = time.time() - data_time
real_cpu = data.to(device)
real_label = label.clone().detach()

netD, netG, netV = get_BNN_models(DATASET, device, LABEL)
net_noise = get_noise(DATASET, device, LABEL)
fed_state_dict = get_netG(netD,netV,net_noise)
netG.load_state_dict(fed_state_dict)
optD = optim.Adam(netD.parameters(), lr1, betas=(0.5, 0.999))
optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))

Acc_results_D = []
Acc_results_G = []


transform = transforms.Compose(
            [transforms.Resize(32),
             transforms.ToTensor(),
             transforms.Normalize(0.5, 0.5)])

trainset = datasets.MNIST(root=ROOT, train=True, download=True, transform=transform)
testset = datasets.MNIST(root=ROOT, train=False, download=True, transform=transform)

testloader = torch.utils.data.DataLoader(
        testset, batch_size=DATA_SAMPLE, shuffle=False, num_workers=WORKERS)

trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=DATA_SAMPLE, shuffle=False, num_workers=WORKERS)
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
n_comp = 10


In [5]:
# golden search algorithm to determined the radius of the sample ball, due to traditional updating netV is too slow
# no meaning for n_dis =2 for our case, just to have output for every update
# sample too much would arise cuda memory error
def gs_radius(radius1,radius2,netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    net_noise = get_noise(DATASET, device, LABEL)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2


def gs_radius2(radius1,radius2, netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    net_noise = get_noise(DATASET, device, LABEL)
    net_noise1 = get_noise(DATASET, device, LABEL)
    net_noise_cauchy = get_cauchy_noise(netD,net_noise,net_noise1)
    net_noise.load_state_dict(net_noise_cauchy)
    #net_noise = get_noise(DATASET, device, LABEL)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2

def gs_radius3(radius1,radius2, netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    net_noise = get_noise_uniform(DATASET, device, LABEL)

    #net_noise = get_noise(DATASET, device, LABEL)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2

In [6]:
# data selected
mean_V = []
n_dis2 = 1
n_dis = 1
mode = 1
gam3 = 1
mcr_gan_loss = MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
loss_V = 0
last_V = -1000
num_steps = 500
lr1 = 1e-3
lr2 = 1e-3
optD = optim.Adam(netD.parameters(), lr1, betas=(0.5, 0.999))
optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
meanV_sum = []
radius_list = []
radius = 1

for epoch in range(num_steps):
    if 1==1:
        
        #radius,_ = gs_radius(radius1=0,radius2=100,netD=netD,real_cpu=real_cpu,real_label=real_label,tor=1e-5,n_dis=2,lam=0.618,mode=mode)
        #radius_list.append(radius)
        #print(radius)

        for i in range(n_dis):
            netD.zero_grad()
            optD.zero_grad()
            Z = netD(real_cpu)
            net_noise = get_noise(DATASET, device, LABEL)
            radius,_ = gs_radius(radius1=0,radius2=100,netD=netD,real_cpu=real_cpu,real_label=real_label,tor=1e-5,n_dis=2,lam=0.618,mode=mode)
            radius_list.append(radius)
            print(radius)
            fed_state_dict = get_netG(netD,netV,net_noise,radius)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errD, errD_EC = mcr_gan_loss(Z, Z_bar, real_label, 1, n_dis)
            print(errD, errD_EC)
            errD.backward()
            optD.step()
        temp = 0
        for j in range(n_dis2):
            netG.zero_grad()
            optG.zero_grad()
            Z = netD(real_cpu)
            net_noise = get_noise(DATASET, device, LABEL)
            radius,_ = gs_radius(radius1=0,radius2=100,netD=netD,real_cpu=real_cpu,real_label=real_label,tor=1e-5,n_dis=2,lam=0.618,mode=mode)
            radius_list.append(radius)
            print(radius)
            fed_state_dict = get_netG(netD,netV,net_noise,radius)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG, errG_EC = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            print(errG, errG_EC)
            temp = errG
            errG = (-1) * errG
            errG.backward()
            optG.step()

            update_dic = update_netV(netV,netG,net_noise,fed_state_dict,radius)
            netV.load_state_dict(update_dic)

        loss_V = temp
        

4.082952496712157
tensor(-50.8248, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-5.6488, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.4272, device='cuda:0', grad_fn=<NegBackward0>), tensor(-40.7488, device='cuda:0', grad_fn=<AddBackward0>)]
7.0778034528674
tensor(-41.6678, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-2.8171, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.3466, device='cuda:0', grad_fn=<NegBackward0>), tensor(-34.5042, device='cuda:0', grad_fn=<AddBackward0>)]
9.031365670307848
tensor(-43.8813, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-2.8171, device='cuda:0', grad_fn=<NegBackward0>), tensor(-4.7955, device='cuda:0', grad_fn=<NegBackward0>), tensor(-36.2687, device='cuda:0', grad_fn=<AddBackward0>)]
4.887804185854826
tensor(-65.5748, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-7.8978, device='cuda:0', grad_fn=<NegBackward0>), tensor(-5.1223, device='cuda:0', grad_fn=<NegBackward0>), tensor(-52.5547, device='cuda:0', grad_fn=<AddBackwar

5.220705839708297
tensor(-97.2889, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-15.1396, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.1281, device='cuda:0', grad_fn=<NegBackward0>), tensor(-75.0212, device='cuda:0', grad_fn=<AddBackward0>)]
5.062379341737056
tensor(-99.5331, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-15.1396, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.0799, device='cuda:0', grad_fn=<NegBackward0>), tensor(-76.3136, device='cuda:0', grad_fn=<AddBackward0>)]
4.8574281368192915
tensor(-100.7774, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-15.5901, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.2865, device='cuda:0', grad_fn=<NegBackward0>), tensor(-76.9008, device='cuda:0', grad_fn=<AddBackward0>)]
4.7881027738110875
tensor(-98.0188, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-15.5901, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.5412, device='cuda:0', grad_fn=<NegBackward0>), tensor(-74.8875, device='cuda:0', grad_fn=<A

5.185607390610618
tensor(-106.8402, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-19.2894, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.7985, device='cuda:0', grad_fn=<NegBackward0>), tensor(-79.7523, device='cuda:0', grad_fn=<AddBackward0>)]
5.110430342931427
tensor(-106.6574, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-19.5465, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.8266, device='cuda:0', grad_fn=<NegBackward0>), tensor(-79.2843, device='cuda:0', grad_fn=<AddBackward0>)]
4.852991125400469
tensor(-107.5190, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-19.5465, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.7333, device='cuda:0', grad_fn=<NegBackward0>), tensor(-79.2393, device='cuda:0', grad_fn=<AddBackward0>)]
4.651029291817718
tensor(-106.0463, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-19.7486, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.8104, device='cuda:0', grad_fn=<NegBackward0>), tensor(-78.4873, device='cuda:0', grad_fn=<

4.545319081744706
tensor(-110.0342, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-22.4863, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.2610, device='cuda:0', grad_fn=<NegBackward0>), tensor(-79.2869, device='cuda:0', grad_fn=<AddBackward0>)]
4.508770407181754
tensor(-109.8535, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-22.4863, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.1290, device='cuda:0', grad_fn=<NegBackward0>), tensor(-79.2382, device='cuda:0', grad_fn=<AddBackward0>)]
4.9200640764483206
tensor(-110.2032, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-22.7050, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.7910, device='cuda:0', grad_fn=<NegBackward0>), tensor(-79.7072, device='cuda:0', grad_fn=<AddBackward0>)]
4.46945360044138
tensor(-110.8611, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-22.7050, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.8076, device='cuda:0', grad_fn=<NegBackward0>), tensor(-79.3486, device='cuda:0', grad_fn=<

4.470608131875812
tensor(-111.8336, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-25.0467, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.6720, device='cuda:0', grad_fn=<NegBackward0>), tensor(-79.1150, device='cuda:0', grad_fn=<AddBackward0>)]
4.313614570562149
tensor(-114.8258, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-25.1482, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.9628, device='cuda:0', grad_fn=<NegBackward0>), tensor(-80.7148, device='cuda:0', grad_fn=<AddBackward0>)]
4.664500796519447
tensor(-114.2708, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-25.1482, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.0635, device='cuda:0', grad_fn=<NegBackward0>), tensor(-81.0591, device='cuda:0', grad_fn=<AddBackward0>)]
4.5136207922536755
tensor(-115.6769, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-25.3207, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.3739, device='cuda:0', grad_fn=<NegBackward0>), tensor(-80.9824, device='cuda:0', grad_fn=

...

4.71757808158553
tensor(-139.1559, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-38.0543, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.9778, device='cuda:0', grad_fn=<NegBackward0>), tensor(-91.1238, device='cuda:0', grad_fn=<AddBackward0>)]
5.144892202352085
tensor(-133.4918, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-38.0077, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.5860, device='cuda:0', grad_fn=<NegBackward0>), tensor(-87.8981, device='cuda:0', grad_fn=<AddBackward0>)]
4.896850161711283
tensor(-132.9961, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-38.0077, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.3016, device='cuda:0', grad_fn=<NegBackward0>), tensor(-87.6868, device='cuda:0', grad_fn=<AddBackward0>)]
4.346812391836395
tensor(-134.7164, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-38.2476, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.0248, device='cuda:0', grad_fn=<NegBackward0>), tensor(-87.4441, device='cuda:0', grad_fn=<A

4.293267939828034
tensor(-139.0891, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-38.3567, device='cuda:0', grad_fn=<NegBackward0>), tensor(-10.0964, device='cuda:0', grad_fn=<NegBackward0>), tensor(-90.6360, device='cuda:0', grad_fn=<AddBackward0>)]
5.138654339020219
tensor(-137.7952, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-38.3567, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.8331, device='cuda:0', grad_fn=<NegBackward0>), tensor(-90.6055, device='cuda:0', grad_fn=<AddBackward0>)]
4.400823138493836
tensor(-133.1286, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-38.3165, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.7722, device='cuda:0', grad_fn=<NegBackward0>), tensor(-87.0400, device='cuda:0', grad_fn=<AddBackward0>)]
4.856338514824593
tensor(-134.1393, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-38.3165, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.7658, device='cuda:0', grad_fn=<NegBackward0>), tensor(-88.0570, device='cuda:0', grad_fn=

4.5585347907986975
tensor(-135.5409, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-38.2012, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.1837, device='cuda:0', grad_fn=<NegBackward0>), tensor(-89.1560, device='cuda:0', grad_fn=<AddBackward0>)]
4.609687942207385
tensor(-133.2578, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-38.3326, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.6548, device='cuda:0', grad_fn=<NegBackward0>), tensor(-87.2705, device='cuda:0', grad_fn=<AddBackward0>)]
5.059868358907418
tensor(-134.9112, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-38.3326, device='cuda:0', grad_fn=<NegBackward0>), tensor(-7.9126, device='cuda:0', grad_fn=<NegBackward0>), tensor(-88.6660, device='cuda:0', grad_fn=<AddBackward0>)]
4.949837103225579
tensor(-139.2439, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-38.3951, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.1867, device='cuda:0', grad_fn=<NegBackward0>), tensor(-91.6620, device='cuda:0', grad_fn=

4.433301224863487
tensor(-136.4384, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-38.1760, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.6148, device='cuda:0', grad_fn=<NegBackward0>), tensor(-88.6476, device='cuda:0', grad_fn=<AddBackward0>)]
5.085135179883219
tensor(-136.2623, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-38.1760, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.5012, device='cuda:0', grad_fn=<NegBackward0>), tensor(-89.5851, device='cuda:0', grad_fn=<AddBackward0>)]
4.586744422941742
tensor(-137.3960, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-38.2908, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.9853, device='cuda:0', grad_fn=<NegBackward0>), tensor(-90.1199, device='cuda:0', grad_fn=<AddBackward0>)]
4.640938843447499
tensor(-136.5087, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-38.2908, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.7754, device='cuda:0', grad_fn=<NegBackward0>), tensor(-89.4425, device='cuda:0', grad_fn=<

4.500024961157179
tensor(-136.4396, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-38.1209, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.8819, device='cuda:0', grad_fn=<NegBackward0>), tensor(-89.4368, device='cuda:0', grad_fn=<AddBackward0>)]
4.4829427441679375
tensor(-140.1128, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-38.1918, device='cuda:0', grad_fn=<NegBackward0>), tensor(-10.4208, device='cuda:0', grad_fn=<NegBackward0>), tensor(-91.5003, device='cuda:0', grad_fn=<AddBackward0>)]
4.563200695275266
tensor(-138.9292, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-38.1918, device='cuda:0', grad_fn=<NegBackward0>), tensor(-9.8988, device='cuda:0', grad_fn=<NegBackward0>), tensor(-90.8386, device='cuda:0', grad_fn=<AddBackward0>)]
4.872663389441908
tensor(-135.8990, device='cuda:0', grad_fn=<NegBackward0>) [tensor(-38.2949, device='cuda:0', grad_fn=<NegBackward0>), tensor(-8.1417, device='cuda:0', grad_fn=<NegBackward0>), tensor(-89.4623, device='cuda:0', grad_fn

In [7]:
radius_list

[4.082952496712157,
 7.0778034528674,
 9.031365670307848,
 4.887804185854826,
 23.146923633172246,
 8.338121295595247,
 4.033539882544882,
 4.286734928234952,
 17.288930171190536,
 4.292865523670321,
 4.233200208073835,
 4.23906262083155,
 4.3374301348851905,
 4.973885543930991,
 4.572307156546224,
 4.8576127188799685,
 4.984115156243226,
 4.853187211261954,
 99.95064099587752,
 4.36914699052067,
 4.531724805362985,
 4.6308991857418045,
 4.427285309298026,
 5.237413501717815,
 4.484291585488522,
 4.845786402950933,
 99.85691287198556,
 4.524535548969006,
 5.123794771175744,
 4.4827548427387285,
 4.52453018889251,
 5.313822067805895,
 5.378551456540364,
 5.220705839708297,
 5.062379341737056,
 4.8574281368192915,
 4.7881027738110875,
 4.995574707544125,
 5.056131475554509,
 5.089397529818009,
 4.822704538050435,
 4.816679455980423,
 5.007373139481357,
 4.626756860837168,
 4.959580150856138,
 5.161209879212568,
 4.767680764448015,
 5.83636292827869,
 4.618543860459796,
 4.978331015544276

In [8]:
#acc at first for NetD
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(testloader, netD, netG)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netD, netG)


print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

#acc at first for NetD
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
print('NetD, Acc at first',count/10000)
Acc_results_D.append(count/10000)

#acc at first for NetG
train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

print("----------------------------")
print("Train_z, Test_z")
nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)


#acc at first
from sklearn.neighbors import KNeighborsClassifier 
knn= KNeighborsClassifier(n_neighbors=10) 

knn.fit(train_Z,train_labels) 
knn.score(test_Z,test_labels) 
 
y_predict = knn.predict(test_Z) 

count =0
for k in range(10000):
    if y_predict[k] == test_labels[k]:
        count +=1
print('NetG, Acc at first',count/10000)
Acc_results_G.append(count/10000)


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.05s/it]


----------------------------
Train_z, Test_z
PCA: 0.9399
SVD: 0.9224
NetD, Acc at first 0.9673


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:01<00:00,  1.99s/it]


----------------------------
Train_z, Test_z
PCA: 0.926
SVD: 0.9243
NetG, Acc at first 0.9545


In [9]:
torch.save(netD, 'MNIST_NetD-mode1-radius2.pth')
torch.save(netV, 'MNIST_NetV-mode1-radius2.pth')
torch.save(netG, 'MNIST_NetG-mode1-radius2.pth')

In [10]:
#robust testing
r_list = [0,0.1,0.2,0.5,1,2,3,4,6,8,10]
sample_num = 20

In [11]:
# sample 20 times of NetG at final radius, NetD is fixed, length 0.1,0.2,0.5,1,2,4,6,8,10,15,20
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample = []
for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[1])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample.append(count/10000)

print(max(Acc_results_G_sample),min(Acc_results_G_sample),np.mean(Acc_results_G_sample),np.var(Acc_results_G_sample))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.12s/it]


----------------------------
Train_z, Test_z
PCA: 0.9337
SVD: 0.9192
NetG_fixD, Acc at sample 1 0.9694


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.02s/it]


----------------------------
Train_z, Test_z
PCA: 0.9331
SVD: 0.9164
NetG_fixD, Acc at sample 2 0.9698


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.14s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.9327
SVD: 0.9176
NetG_fixD, Acc at sample 18 0.9695


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.36s/it]


----------------------------
Train_z, Test_z
PCA: 0.9338
SVD: 0.918
NetG_fixD, Acc at sample 19 0.9696


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.26s/it]


----------------------------
Train_z, Test_z
PCA: 0.9327
SVD: 0.9175
NetG_fixD, Acc at sample 20 0.97
0.9701 0.9693 0.9697049999999999 5.047499999998888e-08


In [12]:
# sample 20 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample2 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[2])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample2.append(count/10000)
    
print(max(Acc_results_G_sample2),min(Acc_results_G_sample2),np.mean(Acc_results_G_sample2),np.var(Acc_results_G_sample2))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.20s/it]


----------------------------
Train_z, Test_z
PCA: 0.933
SVD: 0.9192
NetG_fixD, Acc at sample 1 0.9692


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.31s/it]


----------------------------
Train_z, Test_z
PCA: 0.9334
SVD: 0.9191
NetG_fixD, Acc at sample 2 0.9697


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.27s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.9319
SVD: 0.9178999999999999
NetG_fixD, Acc at sample 18 0.9701


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.13s/it]


----------------------------
Train_z, Test_z
PCA: 0.9338
SVD: 0.9173
NetG_fixD, Acc at sample 19 0.9698


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.08s/it]


----------------------------
Train_z, Test_z
PCA: 0.9336
SVD: 0.9194
NetG_fixD, Acc at sample 20 0.9693
0.9707 0.969 0.9696 1.4899999999999605e-07


In [13]:
# sample 20 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample3 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[3])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample3.append(count/10000)
    
print(max(Acc_results_G_sample3),min(Acc_results_G_sample3),np.mean(Acc_results_G_sample3),np.var(Acc_results_G_sample3))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.12s/it]


----------------------------
Train_z, Test_z
PCA: 0.9321
SVD: 0.9211
NetG_fixD, Acc at sample 1 0.9692


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.06s/it]


----------------------------
Train_z, Test_z
PCA: 0.9315
SVD: 0.9206
NetG_fixD, Acc at sample 2 0.9687


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.15s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.9332
SVD: 0.9207
NetG_fixD, Acc at sample 18 0.9695


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.14s/it]


----------------------------
Train_z, Test_z
PCA: 0.9327
SVD: 0.9188000000000001
NetG_fixD, Acc at sample 19 0.9699


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.06s/it]


----------------------------
Train_z, Test_z
PCA: 0.9339999999999999
SVD: 0.9212
NetG_fixD, Acc at sample 20 0.9697
0.9703 0.9684 0.9693250000000001 2.578750000000012e-07


In [14]:
# sample 20 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample4 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[4])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample4.append(count/10000)
    
print(max(Acc_results_G_sample4),min(Acc_results_G_sample4),np.mean(Acc_results_G_sample4),np.var(Acc_results_G_sample4))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.09s/it]


----------------------------
Train_z, Test_z
PCA: 0.9359999999999999
SVD: 0.9285
NetG_fixD, Acc at sample 1 0.9695


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.08s/it]


----------------------------
Train_z, Test_z
PCA: 0.9355
SVD: 0.9244
NetG_fixD, Acc at sample 2 0.9685


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.11s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.9354
SVD: 0.9248
NetG_fixD, Acc at sample 18 0.9684


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.08s/it]


----------------------------
Train_z, Test_z
PCA: 0.9339999999999999
SVD: 0.9243
NetG_fixD, Acc at sample 19 0.9675


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.13s/it]


----------------------------
Train_z, Test_z
PCA: 0.9303
SVD: 0.9229
NetG_fixD, Acc at sample 20 0.9675
0.9695 0.9665 0.96806 7.084000000000061e-07


In [15]:
# sample 20 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample5 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[5])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample5.append(count/10000)
    
print(max(Acc_results_G_sample5),min(Acc_results_G_sample5),np.mean(Acc_results_G_sample5),np.var(Acc_results_G_sample5))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.17s/it]


----------------------------
Train_z, Test_z
PCA: 0.9325
SVD: 0.9259
NetG_fixD, Acc at sample 1 0.9602


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.12s/it]


----------------------------
Train_z, Test_z
PCA: 0.9316
SVD: 0.9227
NetG_fixD, Acc at sample 2 0.9606


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.08s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.9254
SVD: 0.9192
NetG_fixD, Acc at sample 18 0.9532


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.07s/it]


----------------------------
Train_z, Test_z
PCA: 0.9315
SVD: 0.9251
NetG_fixD, Acc at sample 19 0.9523


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.05s/it]


----------------------------
Train_z, Test_z
PCA: 0.9352
SVD: 0.9286
NetG_fixD, Acc at sample 20 0.9617
0.9638 0.9518 0.958175 9.865875000000008e-06


In [16]:
# sample 20 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample6 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[6])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample6.append(count/10000)
    
print(max(Acc_results_G_sample6),min(Acc_results_G_sample6),np.mean(Acc_results_G_sample6),np.var(Acc_results_G_sample6))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.15s/it]


----------------------------
Train_z, Test_z
PCA: 0.9251
SVD: 0.9196
NetG_fixD, Acc at sample 1 0.9119


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.01s/it]


----------------------------
Train_z, Test_z
PCA: 0.927
SVD: 0.9249
NetG_fixD, Acc at sample 2 0.847


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.07s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.9332
SVD: 0.9283
NetG_fixD, Acc at sample 18 0.8209


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.09s/it]


----------------------------
Train_z, Test_z
PCA: 0.9281
SVD: 0.9207
NetG_fixD, Acc at sample 19 0.8731


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.03s/it]


----------------------------
Train_z, Test_z
PCA: 0.9256
SVD: 0.9189
NetG_fixD, Acc at sample 20 0.8815
0.9221 0.783 0.8478299999999999 0.0017780071


In [17]:
# sample 20 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample7 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[7])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample7.append(count/10000)
    
print(max(Acc_results_G_sample7),min(Acc_results_G_sample7),np.mean(Acc_results_G_sample7),np.var(Acc_results_G_sample7))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.09s/it]


----------------------------
Train_z, Test_z
PCA: 0.9278
SVD: 0.9255
NetG_fixD, Acc at sample 1 0.6299


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.11s/it]


----------------------------
Train_z, Test_z
PCA: 0.9198
SVD: 0.9142
NetG_fixD, Acc at sample 2 0.4727


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.07s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.9242
SVD: 0.9185
NetG_fixD, Acc at sample 18 0.3743


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.11s/it]


----------------------------
Train_z, Test_z
PCA: 0.9275
SVD: 0.9247
NetG_fixD, Acc at sample 19 0.2644


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.10s/it]


----------------------------
Train_z, Test_z
PCA: 0.9299999999999999
SVD: 0.9255
NetG_fixD, Acc at sample 20 0.3476
0.6299 0.2633 0.40830500000000003 0.009744182475000002


In [18]:
# sample 20 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample8 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[8])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample8.append(count/10000)
    
print(max(Acc_results_G_sample8),min(Acc_results_G_sample8),np.mean(Acc_results_G_sample8),np.var(Acc_results_G_sample8))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.16s/it]


----------------------------
Train_z, Test_z
PCA: 0.9238999999999999
SVD: 0.9225
NetG_fixD, Acc at sample 1 0.113


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.07s/it]


----------------------------
Train_z, Test_z
PCA: 0.9203
SVD: 0.9228
NetG_fixD, Acc at sample 2 0.1595


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.12s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.9225
SVD: 0.9182
NetG_fixD, Acc at sample 18 0.1343


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.09s/it]


----------------------------
Train_z, Test_z
PCA: 0.9308
SVD: 0.9261
NetG_fixD, Acc at sample 19 0.1237


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.10s/it]


----------------------------
Train_z, Test_z
PCA: 0.9266
SVD: 0.924
NetG_fixD, Acc at sample 20 0.1045
0.1788 0.1045 0.13327999999999998 0.0005036315999999999


In [19]:
# sample 20 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample9 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[9])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample9.append(count/10000)
    
print(max(Acc_results_G_sample9),min(Acc_results_G_sample9),np.mean(Acc_results_G_sample9),np.var(Acc_results_G_sample9))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.11s/it]


----------------------------
Train_z, Test_z
PCA: 0.9272
SVD: 0.9269000000000001
NetG_fixD, Acc at sample 1 0.1454


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.15s/it]


----------------------------
Train_z, Test_z
PCA: 0.9314
SVD: 0.9254
NetG_fixD, Acc at sample 2 0.0995


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.08s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.9268
SVD: 0.9234
NetG_fixD, Acc at sample 18 0.1072


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.07s/it]


----------------------------
Train_z, Test_z
PCA: 0.9233
SVD: 0.919
NetG_fixD, Acc at sample 19 0.1173


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.20s/it]


----------------------------
Train_z, Test_z
PCA: 0.9336
SVD: 0.9307
NetG_fixD, Acc at sample 20 0.1056
0.1454 0.0822 0.10841 0.00018399490000000005


In [20]:
# sample 20 times of NetG at final radius, NetD is fixed
from sklearn.neighbors import KNeighborsClassifier 
Acc_results_G_sample10 = []

for _ in range(sample_num):
    #acc at first
    net_noise = get_noise(DATASET, device, LABEL)
    fed_state_dict = get_netG(netD,netV,net_noise,r_list[10])
    netG.load_state_dict(fed_state_dict)
    
    train_X, train_Z, train_X_bar, train_Z_bar, train_labels = extract_features2(trainloader, netG, netD)

    test_X, test_Z, test_X_bar, test_Z_bar, test_labels = extract_features2(testloader, netG, netD)

    print("----------------------------")
    print("Train_z, Test_z")
    nearsub(n_comp, train_Z, train_labels, test_Z, test_labels)

    knn= KNeighborsClassifier(n_neighbors=10) 

    knn.fit(train_Z,train_labels) 
    knn.score(test_Z_bar,test_labels) 

    y_predict = knn.predict(test_Z_bar) 

    count =0
    for k in range(10000):
        if y_predict[k] == test_labels[k]:
            count +=1
    print('NetG_fixD, Acc at sample',_+1,count/10000)
    Acc_results_G_sample10.append(count/10000)
    
print(max(Acc_results_G_sample10),min(Acc_results_G_sample10),np.mean(Acc_results_G_sample10),np.var(Acc_results_G_sample10))

extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.05s/it]


----------------------------
Train_z, Test_z
PCA: 0.9358
SVD: 0.9267
NetG_fixD, Acc at sample 1 0.1021


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.12s/it]


----------------------------
Train_z, Test_z
PCA: 0.9307
SVD: 0.9244
NetG_fixD, Acc at sample 2 0.1081


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.11s/it]


...

----------------------------
Train_z, Test_z
PCA: 0.9297
SVD: 0.9275
NetG_fixD, Acc at sample 18 0.1099


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.11s/it]


----------------------------
Train_z, Test_z
PCA: 0.9299
SVD: 0.9278
NetG_fixD, Acc at sample 19 0.1042


extracting all features from dataset: 100%|██████████████████████████████████████████████| 1/1 [00:02<00:00,  2.06s/it]


----------------------------
Train_z, Test_z
PCA: 0.9285
SVD: 0.9252
NetG_fixD, Acc at sample 20 0.0944
0.1193 0.0791 0.10293 0.00012730110000000005
